<a href="https://colab.research.google.com/github/anvargasch/IEStoPBRT/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IES → EXR converter for PBRT (Type C)

This notebook converts an **IES Type C (LM-63)** photometric file into a **normalized lat-long EXR** suitable for **PBRT v4 goniometric lights**.

- Angularly correct interpolation (no resampling artifacts)
- Solid-angle normalization (∫ f dΩ = 1)
- No assumptions about luminous flux


## 1. Clone the repository

In [ ]:
!git clone https://github.com/anvargasch/IEStoPBRT.git
%cd IEStoPBRT

## 2. Install dependencies

In [ ]:
!pip install numpy pyexr matplotlib

## 3. Import the converter

In [ ]:
import sys
sys.path.append('/content/IEStoPBRT')

import importlib
import ies2exr
importlib.reload(ies2exr)

## 4. Convert an IES file to EXR

In [ ]:
filename = 'IESfiles/IESsiemens.ies'   # replace with your IES file
output   = 'IESsiemens_latlong.exr'

meta = ies2exr.ies_to_exr(filename, output, size=1024)

print('Conversion completed.')
print('Metadata:')
for k, v in meta.items():
    print(f'  {k}: {v}')

## 5. Visualize the generated EXR (R channel)

In [ ]:
import pyexr
import numpy as np
import matplotlib.pyplot as plt

img = pyexr.read(output)
r = img[:, :, 0]

plt.figure(figsize=(6, 6))
plt.imshow(r, cmap='gray')
plt.title('Normalized angular intensity (lat-long)')
plt.colorbar(label='Relative intensity')
plt.axis('off')
plt.show()

## 6. Usage in PBRT

The generated EXR represents a **normalized angular distribution**.

In PBRT v4, use it as a `goniometric` light source and specify the **nominal luminous flux** directly:

```pbrt
LightSource "goniometric"
    "string filename" ["IESsiemens_latlong.exr"]
    "blackbody I" [2700]
    "float power" [<nominal lumens>]
```

No additional scaling is applied inside the EXR.
